# Pregunta 3: Desafío Kaggle

## Obtención de los datos

In [0]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle
!wget -O ~/.kaggle/kaggle.json http://vps.csaldias.cl/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

--2018-08-30 22:45:26--  http://vps.csaldias.cl/kaggle.json
Resolving vps.csaldias.cl (vps.csaldias.cl)... 170.239.86.99
Connecting to vps.csaldias.cl (vps.csaldias.cl)|170.239.86.99|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64 [application/json]
Saving to: ‘/root/.kaggle/kaggle.json’

/root/.kaggle/kaggl 100%[===================>]      64  --.-KB/s    in 0s      

2018-08-30 22:45:26 (7.17 MB/s) - ‘/root/.kaggle/kaggle.json’ saved [64/64]



In [3]:
!kaggle competitions download -c tareaANN

  0%|                                               | 0.00/11.9k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 11.9k/11.9k [00:00<00:00, 7.36MB/s]
  0%|                                               | 0.00/2.25k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 2.25k/2.25k [00:00<00:00, 3.13MB/s]
 96%|██████████████████████████████████████▌ | 339M/352M [00:04<00:00, 81.5MB/s]
100%|████████████████████████████████████████| 352M/352M [00:04<00:00, 81.9MB/s]
100%|█████████████████████████████████████▉| 1.37G/1.37G [00:19<00:00, 81.4MB/s]
100%|██████████████████████████████████████| 1.37G/1.37G [00:19<00:00, 76.3MB/s]
 79%|█████████████████████████████▊        | 13.0M/16.5M [00:00<00:00, 16.4MB/s]
100%|██████████████████████████████████████| 16.5M/16.5M [00:00<00:00, 27.7MB/s]
 88%|█████████████████████████████████▍    | 58.0M/66.0M [00:01<00:00, 30.7MB/s]
100%|██████████████████████████████████████| 66.0M/66.0M [00:01<00:00, 55.3MB/s]


In [4]:
!ls -lh

total 1.8G
-rw-r--r-- 1 root root  17M Aug 30 22:46 frames_test.tar.gz
-rw-r--r-- 1 root root  66M Aug 30 22:46 frames_train.tar.gz
-rw-r--r-- 1 root root 352M Aug 30 22:45 images_test.npy
-rw-r--r-- 1 root root 1.4G Aug 30 22:45 images_train.npy
-rw-r--r-- 1 root root  12K Aug 30 22:45 labels_train.csv
drwxr-xr-x 2 root root 4.0K Aug 29 17:36 sample_data
-rw-r--r-- 1 root root 2.3K Aug 30 22:45 sample_submision.csv


## Carga y limpieza de los datos

In [0]:
import numpy as np
import pandas as pd

x_train = np.load('images_train.npy')
df_train = pd.read_csv('labels_train.csv')

In [17]:
from keras.utils import to_categorical
from skimage.transform import rescale

x_train_rescaled = []
for image in x_train:
  x_train_rescaled.append(rescale(image, 1.0 / 4.0))
x_train_rescaled = np.asarray(x_train_rescaled)

y_train = to_categorical(df_train['count'], 51)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [3]:
x_train_rescaled.shape

(1600, 120, 160, 3)

## Definición de la CNN

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm

In [0]:
model = Sequential()

model.add(Conv2D(128, (5, 5), activation='relu', padding='same', input_shape=x_train_rescaled.shape[1:]))
model.add(Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(51, activation='softmax'))

In [90]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 120, 160, 128)     9728      
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 60, 80, 64)        73792     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 30, 40, 64)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 30, 40, 64)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 30, 40, 64)        36928     
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 15, 20, 64)        36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 7, 10, 64)         0         
__________

## Entrenamiento de la CNN

In [0]:
from keras.optimizers import SGD, rmsprop, Adam
import math

In [0]:
epochs=25
learning_rate = 0.5
decay_rate = 1e-6

opt = rmsprop(lr=0.001, decay=1e-6)
#opt = Adam(lr = 0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=opt, loss=root_mean_squared_error, metrics=['accuracy'])

In [95]:
batch_size=32
history=model.fit(x_train_rescaled, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, shuffle=True, verbose=2)

Train on 1440 samples, validate on 160 samples
Epoch 1/25
 - 9s - loss: 3.3232 - acc: 0.0667 - val_loss: 3.0450 - val_acc: 0.1562
Epoch 2/25
 - 8s - loss: 3.1892 - acc: 0.0778 - val_loss: 2.9085 - val_acc: 0.0813
Epoch 3/25
 - 8s - loss: 3.0598 - acc: 0.0924 - val_loss: 2.8676 - val_acc: 0.1313
Epoch 4/25
 - 8s - loss: 2.9096 - acc: 0.1215 - val_loss: 2.9554 - val_acc: 0.0563
Epoch 5/25
 - 8s - loss: 2.7275 - acc: 0.1604 - val_loss: 2.8011 - val_acc: 0.1000
Epoch 6/25
 - 8s - loss: 2.5890 - acc: 0.1944 - val_loss: 2.9362 - val_acc: 0.0625
Epoch 7/25
 - 8s - loss: 2.3802 - acc: 0.2562 - val_loss: 3.0808 - val_acc: 0.0500
Epoch 8/25
 - 8s - loss: 2.0706 - acc: 0.3292 - val_loss: 3.8141 - val_acc: 0.0688
Epoch 9/25
 - 8s - loss: 1.7601 - acc: 0.4194 - val_loss: 3.4674 - val_acc: 0.0750
Epoch 10/25
 - 8s - loss: 1.4394 - acc: 0.5347 - val_loss: 3.8168 - val_acc: 0.1000
Epoch 11/25
 - 8s - loss: 1.0916 - acc: 0.6424 - val_loss: 4.8081 - val_acc: 0.0688
Epoch 12/25
 - 8s - loss: 0.8191 - acc

## Pruebas sobre la Red

In [0]:
Y_trained = model.predict(x_train_rescaled,batch_size=1)

In [0]:
y_trained_argmax = []
y_argmax = []
for elem in Y_trained:
  y_trained_argmax.append(np.argmax(elem))
for elem in y_train:
  y_argmax.append(np.argmax(elem))

In [98]:
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_argmax, y_trained_argmax))

1.6723860200324565

## Preparación de Submission a Kaggle

In [68]:
x_test = np.load('images_test.npy')
x_test_rescaled = []
for image in x_test:
  x_test_rescaled.append(rescale(image, 1.0 / 4.0))
x_test_rescaled = np.asarray(x_test_rescaled)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
Y_trained = model.predict(x_test_rescaled, batch_size=1)

In [0]:
y_trained_argmax = []
for elem in Y_trained:
  y_trained_argmax.append(np.argmax(elem))

In [0]:
test_ids = list(range(1, 401))
d = {'id': test_ids, 'count': y_trained_argmax}
entrega = pd.DataFrame(data=d,columns=['id','count'])
entrega.to_csv('mysubmission.csv', index=False)